In [1]:
# 請先確認您的 sklearn 版本是否相同, 如果版本不是 0.21.1 版, 有可能跑出與本範例不同的結果
import sklearn
sklearn.__version__

'0.21.3'

In [2]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = 'C:\\Users\\Rubio\\AppData\\Roaming\\SPB_16.6\\'
df = pd.read_csv(data_path + 'house_train.csv.gz')

train_Y = np.log1p(df['SalePrice'])
df = df.drop(['Id', 'SalePrice'] , axis=1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
# 生活總面積(GrLivArea)對販售條件(SaleCondition)做群聚編碼
# 寫法類似均值編碼, 只是對另一個特徵, 而非目標值
df['SaleCondition'] = df['SaleCondition'].fillna('None')
mean_df = df.groupby(['SaleCondition'])['GrLivArea'].mean().reset_index()
mode_df = df.groupby(['SaleCondition'])['GrLivArea'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['SaleCondition'])['GrLivArea'].median().reset_index()
max_df = df.groupby(['SaleCondition'])['GrLivArea'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, median_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, max_df, how='left', on=['SaleCondition'])
temp.columns = ['SaleCondition', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']
temp

,SaleCondition,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,Abnorml,1436.128713,864,1302.0,4476
1,AdjLand,1112.500000,980,1143.0,1184
2,Alloca,1701.750000,1535,1439.5,3194
3,Family,1480.950000,948,1390.5,2526
4,Normal,1492.968280,864,1456.0,4316
5,Partial,1795.696000,1456,1646.0,5642


In [4]:
mode_df

,SaleCondition,GrLivArea
0,Abnorml,864
1,AdjLand,980
2,Alloca,1535
3,Family,948
4,Normal,864
5,Partial,1456


In [5]:
df = pd.merge(df, temp, how='left', on=['SaleCondition'])
df = df.drop(['SaleCondition'] , axis=1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,2,2008,WD,1492.968280,864,1456.0,4316
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,NaN,0,5,2007,WD,1492.968280,864,1456.0,4316
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,0,9,2008,WD,1492.968280,864,1456.0,4316
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,NaN,0,2,2006,WD,1436.128713,864,1302.0,4476
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,0,12,2008,WD,1492.968280,864,1456.0,4316


In [6]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

40 Numeric Features : ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']



,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,2,2008,1492.968280,864,1456.0,4316
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,5,2007,1492.968280,864,1456.0,4316
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,9,2008,1492.968280,864,1456.0,4316
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,2,2006,1436.128713,864,1302.0,4476
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,12,2008,1492.968280,864,1456.0,4316


In [8]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.846643681071632

In [9]:
# 新特徵 + 線性迴歸 : 有些微改善
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8541521811013844

In [10]:
# 原始特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df_minus)
estimator = RandomForestRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8464557291889392

In [11]:
# 新特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8486167835324261

In [56]:
# HW1
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'C:\\Users\\Rubio\\AppData\\Roaming\\SPB_16.6\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [57]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Cabin'] = df['Cabin'].fillna('NaN')
mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Cabin'])['Fare'].median().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
min_df = df.groupby(['Cabin'])['Fare'].min().reset_index()
count_df = df.groupby(['Cabin'])['Fare'].count().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
temp.columns = ['Cabin', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp

,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,A10,40.125000,40.1250,40.12500,40.1250
1,A14,52.000000,52.0000,52.00000,52.0000
2,A16,39.600000,39.6000,39.60000,39.6000
3,A19,26.000000,26.0000,26.00000,26.0000
4,A20,56.929200,56.9292,56.92920,56.9292
...,...,...,...,...,...
143,F38,7.750000,7.7500,7.75000,7.7500
144,F4,39.000000,39.0000,39.00000,39.0000
145,G6,13.581250,10.4625,13.58125,16.7000
146,NaN,19.157325,8.0500,10.50000,512.3292


In [58]:
temp1 = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp1 = pd.merge(temp1, median_df, how='left', on=['Cabin'])
temp1 = pd.merge(temp1, min_df, how='left', on=['Cabin'])
temp1.columns = ['Cabin', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Min']
temp1

,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Min
0,A10,40.125000,40.1250,40.12500,40.1250
1,A14,52.000000,52.0000,52.00000,52.0000
2,A16,39.600000,39.6000,39.60000,39.6000
3,A19,26.000000,26.0000,26.00000,26.0000
4,A20,56.929200,56.9292,56.92920,56.9292
...,...,...,...,...,...
143,F38,7.750000,7.7500,7.75000,7.7500
144,F4,39.000000,39.0000,39.00000,39.0000
145,G6,13.581250,10.4625,13.58125,10.4625
146,NaN,19.157325,8.0500,10.50000,0.0000


In [59]:
temp2 = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp2 = pd.merge(temp2, median_df, how='left', on=['Cabin'])
temp2 = pd.merge(temp2, count_df, how='left', on=['Cabin'])
temp2.columns = ['Cabin', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Count']
temp2

,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Count
0,A10,40.125000,40.1250,40.12500,1
1,A14,52.000000,52.0000,52.00000,1
2,A16,39.600000,39.6000,39.60000,1
3,A19,26.000000,26.0000,26.00000,1
4,A20,56.929200,56.9292,56.92920,1
...,...,...,...,...,...
143,F38,7.750000,7.7500,7.75000,1
144,F4,39.000000,39.0000,39.00000,2
145,G6,13.581250,10.4625,13.58125,4
146,NaN,19.157325,8.0500,10.50000,687


In [60]:
df1 = df
df2 = df

In [61]:
# 新資料(合併過) use temp
df = pd.merge(df, temp, how='left', on=['Cabin'])
df = df.drop(['Cabin'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,512.3292


In [62]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292


In [63]:
# 原始資料
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)
df.head()

,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292


In [64]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression(solver = 'lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7004864906462793

In [65]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6937385696153646

In [66]:
# 新資料(合併過) use temp1
df1 = pd.merge(df1, temp1, how='left', on=['Cabin'])
df1 = df1.drop(['Cabin'] , axis=1)
df1.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,0.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,0.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,0.0000


In [67]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df1.dtypes, df1.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df1 = df1[num_features]
df1 = df1.fillna(-1)
df1.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Min']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Min
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,0.0000
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,0.0000
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,0.0000


In [68]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df1)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7027336816575153

In [69]:
# 新資料(合併過) use temp2
df2 = pd.merge(df2, temp2, how='left', on=['Cabin'])
df2 = df2.drop(['Cabin'] , axis=1)
df2.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,687
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,687
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,687


In [70]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df2.dtypes, df2.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df2 = df2[num_features]
df2 = df2.fillna(-1)
df2.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Count']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Count
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,687
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,1
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,687
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,2
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,687


In [71]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df2)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6926212511796104

# 總結
發現使用 min 的效果 > max > count (使用 Cabin 的話)

但有可能不適合使用 Cabin 這資料做 groupby，因為資料的 nunique 太多，可以嘗試使用 Pclass or Embarked 看看